In [1]:
import pandas as pd

class GroceryStore:
    def __init__(self, inventory_df):
        self.inventory = inventory_df

    def display_inventory(self):
        print("Inventory:")
        print(self.inventory)

    def update_inventory(self, cart):
        for index, row in cart.iterrows():
            serial = row['Serial']
            quantity = row['Quantity']
            self.inventory.loc[self.inventory['Serial'] == serial, 'Quantity'] -= quantity


class Customer:
    def __init__(self, store):
        self.store = store
        self.cart = pd.DataFrame(columns=['Serial', 'Item', 'Price', 'Quantity'])

    def view_products(self):
        print("Inventory:")
        print(self.store.inventory[['Serial', 'Item', 'Price']])

    def add_to_cart(self):
        try:
            while True:
                serial = int(input("Enter the serial number of the item you want to purchase (0 to stop): "))
                if serial == 0:
                    break
                if serial not in self.store.inventory['Serial'].tolist():
                    print("Invalid serial number!")
                    continue
                quantity = int(input(f"How many {self.store.inventory.loc[self.store.inventory['Serial'] == serial, 'Item'].values[0]}s do you want to purchase? "))
                if quantity < 0:
                    print("Invalid quantity!")
                    continue
                if quantity > self.store.inventory.loc[self.store.inventory['Serial'] == serial, 'Quantity'].values[0]:
                    print("Sorry, the item is either not available or insufficient quantity.")
                    continue
                item_data = self.store.inventory.loc[self.store.inventory['Serial'] == serial].iloc[0]
                item_data['Quantity'] = quantity
                self.cart = self.cart.append(item_data, ignore_index=True)
                print(f"{quantity} {item_data['Item']}(s) added to your cart.")
        except ValueError:
            print("Invalid input! Please enter a valid quantity.")

    def view_cart(self):
        print("Your Cart:")
        print(self.cart)

    def checkout(self):
        total_cost = sum(self.cart['Price'] * self.cart['Quantity'])
        total_cost *= 1.06  # 6% tax
        print("\n\nYour Bill:")
        for index, row in self.cart.iterrows():
            item_name = row['Item']
            price = row['Price']
            quantity = row['Quantity']
            print(f"{item_name}: {quantity} x ${price} = ${price * quantity}")
        print(f"Total Cost (including tax): ${total_cost}")
        self.store.update_inventory(self.cart)

    def display_exit_message(self):
        print("Thank you for shopping with us!")


class Owner:
    def __init__(self, store):
        self.store = store

    def manage_inventory(self):
        print("\nWelcome to the owner view of the grocery store!")
        print("Inventory:")
        print(self.store.inventory)


def main():
    # Read inventory from CSV file
    inventory_df = pd.read_csv('owner_inventory.csv')

    # Create grocery store instance
    store = GroceryStore(inventory_df)

    while True:
        print("\nSelect your view:")
        print("1. Customer view")
        print("2. Owner view")
        print("3. Exit")
        choice = input("Enter your choice (1, 2, or 3): ")

        if choice == '1':
            customer = Customer(store)
            while True:
                print("\nCustomer Options:")
                print("1. View products")
                print("2. Add to cart")
                print("3. View cart")
                print("4. Checkout")
                print("5. Back to main menu")
                option = input("Enter your choice: ")
                if option == '1':
                    customer.view_products()
                elif option == '2':
                    customer.add_to_cart()
                elif option == '3':
                    customer.view_cart()
                elif option == '4':
                    customer.checkout()
                    customer.display_exit_message()
                    break
                elif option == '5':
                    break
                else:
                    print("Invalid option! Please try again.")

        elif choice == '2':
            owner = Owner(store)
            owner.manage_inventory()

        elif choice == '3':
            print("Exiting the program. Goodbye!")
            break

        else:
            print("Invalid choice! Please enter '1', '2', or '3'.")


if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'owner_inventory.csv'